In [1]:
from cods.od.data import MSCOCODataset
from cods.od.models import YOLOModel, DETRModel
import logging
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = (
    "0"  # chose the GPU. If only one, then "0"
)

logging.getLogger().setLevel(logging.INFO)

In [2]:
# set [COCO_PATH] to the directory to your local copy of the COCO dataset
COCO_PATH = "/datasets/shared_datasets/coco/"

data = MSCOCODataset(root=COCO_PATH, split="val")

In [3]:
calibration_ratio = (
    0.5  # set 0.5 to use 50% for calibration and 50% for testing
)

use_smaller_subset = True  # TODO: Temp

if use_smaller_subset:
    data_cal, data_val = data.random_split(
        calibration_ratio, shuffled=False, n_calib_test=800
    )
else:
    data_cal, data_val = data.random_split(calibration_ratio, shuffled=False)

# model and weights are downloaded from https://github.com/facebookresearch/detr
# detr = DETRModel(model_name="detr_resnet50", pretrained=True)
yolo = YOLOModel(model_name="yolov8x.pt", pretrained=True)


print(f"{len(data) = }")
print(f"{len(data_cal) = }")
print(f"{len(data_val) = }")

len(data) = 5000
len(data_cal) = 400
len(data_val) = 400


In [4]:
preds_cal = yolo.build_predictions(
    data_cal,
    dataset_name="mscoco",
    split_name="cal",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=True,
)
preds_val = yolo.build_predictions(
    data_val,
    dataset_name="mscoco",
    split_name="test",
    batch_size=12,
    collate_fn=data._collate_fn,
    shuffle=False,
    force_recompute=True,
)

Force recompute is set to True, building predictions...


  0%|          | 0/34 [00:00<?, ?it/s]

torch.Size([640, 640])
torch.Size([640, 640])


  6%|▌         | 2/34 [00:01<00:19,  1.65it/s]

torch.Size([640, 640])
torch.Size([640, 640])


...

torch.Size([640, 640])


 94%|█████████▍| 32/34 [00:06<00:00,  5.20it/s]

torch.Size([640, 640])


100%|██████████| 34/34 [00:06<00:00,  5.30it/s]

torch.Size([640, 640])
Saving predictions to ./saved_predictions/yolov8x.pt/mscoco/test/predictions_object_detection.pkl
File ./saved_predictions/yolov8x.pt/mscoco/test/predictions_object_detection.pkl already exists, overwriting it


In [5]:
from cods.od.visualization import plot_preds

plot_preds(preds_val, None, 11)

AttributeError: 'NoneType' object has no attribute 'conf_boxes'